In [ ]:
pip install mip

In [ ]:
pip install pysmps

In [3]:
import numpy as np
import mip
from math import *
import pysmps
from pysmps import smps_loader as smps

import sys
sys.setrecursionlimit(10000)


In [5]:
erro = 0.01


def CuttingPlane(N, S):
    # Resolver o problema relaxado
    N.optimize(relax=True)
    print(N.objective_value)

    # Ver se é factível
    if N.status.value:
        print("Não é factível")
        return S  # Se não for factível: Retornar vetor de soluções

    # Ver se a solução é inteira, se for, adicionar ao vetor S e retornar S
    V = np.array([N.vars[i].x for i in range(len(N.vars))])
    W = np.array([round(V[i]) for i in range(len(V))])
    if np.linalg.norm(abs(V - W)) <= erro:
        S = np.insert(S, S.size, N.objective_value)
        return S

    else:
        # Gerar corte para o modelo e resolver de novo
        C = N.generate_cuts([mip.CutType.GOMORY])

        if C.cuts:
            N += C

        S = CuttingPlane(N, S)

    return S
